In [1]:
import random
from collections import deque
from queue import *

from statistics import mean
from collections import deque

class Queue:
    def __init__(self):
        # Contadores estadísticos:
        self.num_custs_delayed: int = None
        self.time_of_delays = None

        # Parámetros de la simulación
        self.mean_interarrival: float = None
        self.mean_service: float = None

        # Lista enlazada con el número de clientes
        # Con la lista enlazada se acelera mucho la velocidad de ejecución
        self.time_arrival = deque()

    def set_parameters(self, mean_interarrival, mean_service):
        self.mean_interarrival = mean_interarrival
        self.mean_service = mean_service

    def enqueue_client(self, sim_time: float):
        self.time_arrival.append(sim_time)

    def attend_person(self, sim_time):
        delay = sim_time - self.time_arrival.popleft()
        self.time_of_delays += delay
        self.num_custs_delayed += 1

    def is_empty(self):
        if len(self.time_arrival) == 0:
            return True
        else:
            return False

    def generate_arrival_time(self):
        return random.expovariate(1/self.mean_interarrival)

    def generate_service_time(self):
        return random.expovariate(1/self.mean_service)

In [ ]:
#Ejercicio 1.7

# Estados del servidor:
IDLE, BUSY = 0, 1

# Eventos del modelo:
ONSITE_ARRIVE = 1
TELEPHONE_ARRIVE = 2
DEPART = 3
MAX_SIM_TIME_REACHED = 4

next_event_type: int
num_events: int
next_events_times = []

# Las dos colas
onsite, telephone = Queue(), Queue()

sim_time: float

# Variables de estado globales
server_status: int

# Parámetros
max_sim_time: float
first_onsite_arrival = 2.0
first_telephone_arrival = 3.0

# Variables del conjunto de simulaciones
onsite_means_delays = deque()
telephone_means_delays = deque()
onsite_attended_clients = deque()
telephone_attended_clientes = deque()
number_of_simulations: int

# Archivos de entrada y salida
onsite_input = (12.0, 6.0)
telephone_input = (10.0, 5.0)
max_sim_time = 480.0
number_of_simulations = 50000

outfile = open("report_1.7.txt", "w+")


def main():
    global num_events
    global next_events_times

    read_input_parameters()

    num_events = 4
    next_events_times = [None] * (num_events + 1)

    print("Ejecutando simulación con", str(number_of_simulations), "repeticiones")
    print("Por favor, espere mientras se ejecuta la simulación.")
    for i in range(number_of_simulations):
        simulate()
        if (i + 1) % 1000 == 0:
            print("Van ", str(i + 1), " repeticiones")

    print("¡Simulación Terminada con éxito!")
    report()


def simulate():

    initialize()

    while True:
        timing()

        if next_event_type == ONSITE_ARRIVE:
            arrive(onsite, next_event_type)

        elif next_event_type == TELEPHONE_ARRIVE:
            arrive(telephone, next_event_type)

        elif next_event_type == DEPART:
            depart()

        if next_event_type == MAX_SIM_TIME_REACHED:
            break

    save_simulation_result()


def initialize():
    global sim_time, server_status

    # Poner el reloj en 0
    sim_time = 0.0

    # Inicializar las variables de estado
    server_status = IDLE
    onsite.time_arrival.clear()
    telephone.time_arrival.clear()

    # Inicializar los contadores estadísticos
    onsite.num_custs_delayed = 0
    onsite.time_of_delays = 0.0
    telephone.num_custs_delayed = 0
    telephone.time_of_delays = 0.0

    # Programar los primeros eventos
    next_events_times[0] = None
    next_events_times[ONSITE_ARRIVE] = first_onsite_arrival
    next_events_times[TELEPHONE_ARRIVE] = first_telephone_arrival
    next_events_times[DEPART] = pow(10, 30)
    next_events_times[MAX_SIM_TIME_REACHED] = max_sim_time


def timing():
    global next_event_type, sim_time
    min_time_next_event = pow(10, 29)
    next_event_type = 0

    # Determinar, en la lista de eventos, cuál es el evento más próximo
    for event_type in range(1, num_events+1):
        if next_events_times[event_type] < min_time_next_event:
            min_time_next_event = next_events_times[event_type]
            next_event_type = event_type

    # En caso de que todos los eventos hayan sido superiores a pow(10, 29) (Caso muy extraño)
    if next_event_type == 0:
        error_message = ' '.join(["¡La lista de eventos está vacía en el momento ", str(sim_time), "!"])
        print(error_message)
        outfile.write(error_message)
        exit(1)

    sim_time = min_time_next_event


def arrive(queue: Queue(), type_of_event: int):
    global server_status
    next_events_times[type_of_event] = sim_time + queue.generate_arrival_time()

    if server_status == BUSY:
        queue.enqueue_client(sim_time)
    else:
        # Como no había nadie en el servidor, se aumenta en 1 la cantidad de clientes que han pasado
        # por la cola, pero no se aumenta su respectivo tiempo de demora, porque no se demoró nada (t=0)
        queue.num_custs_delayed += 1
        server_status = BUSY
        next_events_times[DEPART] = sim_time + queue.generate_service_time()


def depart():
    global server_status
    if onsite.is_empty():
        if telephone.is_empty():
            server_status = IDLE
            next_events_times[DEPART] = pow(10, 30)
        else:
            telephone.attend_person(sim_time)
            next_events_times[DEPART] = sim_time + telephone.generate_service_time()
    else:
        onsite.attend_person(sim_time)
        next_events_times[DEPART] = sim_time + onsite.generate_service_time()


def save_simulation_result():
    onsite_mean_delay = onsite.time_of_delays / onsite.num_custs_delayed  # Promedio de las demoras presenciales
    try:
        telephone_mean_delay = telephone.time_of_delays / telephone.num_custs_delayed  # Promedio de las demoras telefónicas
    except ZeroDivisionError:
        #  Probablemente no se pudo atender a ningún cliente. Es decir, siempre todos estuvieron en cola
        telephone_mean_delay = max_sim_time
    ons_attended_clients = onsite.num_custs_delayed # Número de clientes atendidos en persona
    tel_attended_clients = telephone.num_custs_delayed # Número de clientes atendidos por telefono

    # Añadir esos resultados a los respectivos vectores que guardan los datos de todas las simulaciones

    onsite_means_delays.append(onsite_mean_delay)
    telephone_means_delays.append(telephone_mean_delay)
    onsite_attended_clients.append(ons_attended_clients)
    telephone_attended_clientes.append(tel_attended_clients)


def read_input_parameters():

    global max_sim_time, number_of_simulations

    mean_interarrival, mean_service = onsite_input[0], onsite_input[1]
    onsite.set_parameters(float(mean_interarrival), float(mean_service))

    mean_interarrival, mean_service = telephone_input[0], telephone_input[1]
    telephone.set_parameters(float(mean_interarrival), float(mean_service))

    max_sim_time = max_sim_time

    number_of_simulations = number_of_simulations


def report():

    ons_mean_of_means_delays = mean(onsite_means_delays)
    tel_mean_of_means_delays = mean(telephone_means_delays)

    print(f"""
    Resultados:
    Con {number_of_simulations} simulaciones ejecutadas, tenemos:
    Medidas de desempeño estimadas:
    Tiempo promedio de espera en la cola presencial: {str(round(ons_mean_of_means_delays, 3))},  minutos
    Tiempo promedio de espera en la cola telefónica: {str(round(tel_mean_of_means_delays, 3))}, minutos
    Parámetros de la simulación:
    Tiempo requerido para terminar la simulación: {str(max_sim_time)}, minutos
    ----------------------------------------------------------------------------------------------------
    Cola Presencial:
    Media de llegada: {str(onsite.mean_interarrival)}
    Media de servicio: {str(onsite.mean_service)}  minutos
    Primera llegada: {str(first_onsite_arrival)} minutos
    ----------------------------------------------------------------------------------------------------
    Cola Telefónica:
    Media de llegada: {str(telephone.mean_interarrival)} minutos
    Media de servicio: {str(telephone.mean_service)} minutos
    Primera llegada: {str(first_telephone_arrival)} minutos""")


if __name__ == '__main__':
    main()


Ejecutando simulación con 50000 repeticiones
Por favor, espere mientras se ejecuta la simulación.
Van  1000  repeticiones
Van  2000  repeticiones
Van  3000  repeticiones
Van  4000  repeticiones
Van  5000  repeticiones
Van  6000  repeticiones
Van  7000  repeticiones
Van  8000  repeticiones
Van  9000  repeticiones
Van  10000  repeticiones
Van  11000  repeticiones
Van  12000  repeticiones
Van  13000  repeticiones
Van  14000  repeticiones
Van  15000  repeticiones
Van  16000  repeticiones
Van  17000  repeticiones
Van  18000  repeticiones
Van  19000  repeticiones
Van  20000  repeticiones
Van  21000  repeticiones
Van  22000  repeticiones
Van  23000  repeticiones
Van  24000  repeticiones
Van  25000  repeticiones
Van  26000  repeticiones
Van  27000  repeticiones
Van  28000  repeticiones
Van  29000  repeticiones
Van  30000  repeticiones
Van  31000  repeticiones
Van  32000  repeticiones
Van  33000  repeticiones
Van  34000  repeticiones
Van  35000  repeticiones
Van  36000  repeticiones
Van  37000 